In [ ]:
import logging
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    # Load data from Unity Catalog tables
    logger.info("Loading data from Unity Catalog tables...")
    customers_df = spark.table("genai_demo.sas.customers")
    transactions_df = spark.table("genai_demo.sas.transactions")

    # Step 1: Filter valid transactions
    logger.info("Filtering valid transactions...")
    valid_txns_df = transactions_df.filter((F.col("Sales") > 0) & (F.col("Product") != ""))

    # Step 2: Calculate Effective Price
    logger.info("Calculating Effective Price...")
    trans_step2_df = valid_txns_df.withColumn("EffectivePrice", F.col("Sales") * (1 - F.col("Discount") / 100))

    # Step 3: Calculate Total Value
    logger.info("Calculating Total Value...")
    trans_step3_df = trans_step2_df.withColumn("TotalValue", F.col("EffectivePrice") * F.col("Quantity"))

    # Step 4: Join with Customers
    logger.info("Joining transactions with customer data...")
    full_data_df = trans_step3_df.join(customers_df, "CustomerID", "left").select(
        trans_step3_df["*"], customers_df["Region"], customers_df["JoinDate"]
    )

    # Step 5: Calculate Tenure Days
    logger.info("Calculating Tenure Days...")
    trans_step5_df = full_data_df.withColumn("TenureDays", F.datediff(F.col("TransDate"), F.col("JoinDate")))

    # Step 6: Assign Tenure Category
    logger.info("Assigning Tenure Category...")
    trans_step6_df = trans_step5_df.withColumn(
        "TenureCategory",
        F.when(F.col("TenureDays") < 180, "New")
        .when(F.col("TenureDays") < 365, "Medium")
        .otherwise("Loyal")
    )

    # Step 7: Flag High Value Transactions
    logger.info("Flagging High Value Transactions...")
    trans_step7_df = trans_step6_df.withColumn("HighValueFlag", F.col("TotalValue") > 2000)

    # Step 8: Assign Product Group
    logger.info("Assigning Product Group...")
    trans_step8_df = trans_step7_df.withColumn(
        "ProductGroup",
        F.when(F.col("Product").isin(["A", "C"]), "Core").otherwise("Non-Core")
    )

    # Step 9: Prepare Final Data
    logger.info("Preparing Final Data...")
    final_data_df = trans_step8_df

    # Step 10: Sort by Product Group
    logger.info("Sorting by Product Group...")
    sorted_final_data_df = final_data_df.orderBy("ProductGroup")

    # Step 11: Standardize Total Value
    logger.info("Standardizing Total Value...")
    window_spec = Window.partitionBy("ProductGroup")
    standardized_df = sorted_final_data_df.withColumn(
        "StandardizedTotalValue",
        (F.col("TotalValue") - F.avg("TotalValue").over(window_spec)) / F.stddev("TotalValue").over(window_spec)
    )

    # Step 12: Detect Outliers
    logger.info("Detecting Outliers...")
    enhanced_final_data_df = standardized_df.withColumn(
        "OutlierFlag", F.abs(F.col("StandardizedTotalValue")) > 2
    )

    # Step 13: Write Final Data to Unity Catalog
    logger.info("Writing final data to Unity Catalog...")
    enhanced_final_data_df.write.format("delta").mode("overwrite").saveAsTable("genai_demo.sas.enhanced_final_data")

    # Step 14: Summary Statistics
    logger.info("Generating Summary Statistics...")
    summary_stats_df = enhanced_final_data_df.groupBy("Region", "ProductGroup").agg(
        F.mean("TotalValue").alias("MeanTotalValue"),
        F.sum("TotalValue").alias("SumTotalValue"),
        F.mean("Quantity").alias("MeanQuantity"),
        F.sum("Quantity").alias("SumQuantity"),
        F.mean("Sales").alias("MeanSales"),
        F.sum("Sales").alias("SumSales")
    )
    summary_stats_df.show()

    # Step 15: Frequency Analysis
    logger.info("Performing Frequency Analysis...")
    freq_analysis_df = enhanced_final_data_df.groupBy("TenureCategory", "Region").count()
    freq_analysis_df.show()

    # Step 16: Correlation Analysis
    logger.info("Performing Correlation Analysis...")
    correlation_df = enhanced_final_data_df.select("Sales", "Discount", "Quantity", "TotalValue").corr()
    print(correlation_df)

    # Step 17: Summary by Outlier Flag
    logger.info("Generating Summary by Outlier Flag...")
    summary_by_outlier_df = enhanced_final_data_df.groupBy("OutlierFlag").agg(
        F.mean("Sales").alias("MeanSales"),
        F.stddev("Sales").alias("StdDevSales"),
        F.mean("TotalValue").alias("MeanTotalValue"),
        F.stddev("TotalValue").alias("StdDevTotalValue"),
        F.mean("Quantity").alias("MeanQuantity"),
        F.stddev("Quantity").alias("StdDevQuantity")
    )
    summary_by_outlier_df.show()

except Exception as e:
    logger.error(f"An error occurred: {e}")
